Valuation for Financial Engineers

## Monte Carlo Methods

Importing the required modules.

In [ ]:
import numpy as np
import pandas as pd

### Normal random variables and histograms

Generate an array of 10,000 samples drawn from a standard normal random variable:

In [ ]:
sample = np.random.normal(0,1,10000)

In [ ]:
sample

Import the module used for plotting.

In [ ]:
import matplotlib.pyplot as plt

Proceed with plotting a histogram of the generated sample:

In [ ]:
plt.hist(sample, bins=20, density=True)
plt.ylabel('density')
plt.xlabel('value')
plt.title('Histogram of the sample')
plt.show()

Calculate the mean and standard deviation of the generated sample:

In [ ]:
mean = np.mean(sample)
standard_dev = np.std(sample)

In [ ]:
mean

In [ ]:
standard_dev

The calculated values are very close to a mean of 0 and a standard deviation of 1. This outcome is expected since the sample size is large.

Overlay the standard normal probability density function (PDF) on the histogram.

In [ ]:
from scipy.stats import norm
x = np.linspace(min(sample), max(sample), 100)
p = norm.pdf(x, 0, 1)

In [ ]:
plt.plot(x, p, 'k')
plt.show()

Putting the curve and the historam together:

In [ ]:
plt.hist(sample, bins=20, density=True)
plt.ylabel('density')
plt.xlabel('value')
plt.title('Histogram of the sample')
plt.plot(x, p, 'k')
plt.show()

As expected. With a large sample our histogram matches the density function pretty much perfectly.

### Simulating a path of an asset

Begin by defining the parameters.

In [ ]:
alpha = 0.05
sigma = 0.1
spot = 1000

Simulate the asset path for 30 days using the formula from the notes.

In [ ]:
current_spot = spot
path = [spot]
for i in range(21):
    z = np.random.normal(0,1,1) # Simulate a standard normal
    S_i = current_spot * np.exp((alpha-0.5*sigma**2)*(1/252) + z*sigma*np.sqrt(1/252)) # Forumla for price (1 day movement)
    path.append(S_i) # Add the simulated price to out list 'path'
    current_spot = S_i # Update the current spot with the newly simulated price

Plot the simulated asset path.

In [ ]:
plt.plot(path)

### Simulating multiple paths

Repeat the simulation process, but generate n paths instead of just one, where n is the desired number of simulations.

In [ ]:
alpha = 0.05
sigma = 0.1
spot = 1000
n = 100000

The list `final_spot_simulations` saves the price simulated at the end of the year in each single simulation. Note that the formula for the final spot (after a year) price is updated with $T=1$.

In [ ]:
final_spot_simulations = []
for i in range(n):
    z = float(np.random.normal(0,1,1)) # We inculde the float function to avoid keeping an array (we convert it to a number)
    final_spot = spot * np.exp((alpha-0.5*sigma**2)*(1) + z*sigma*np.sqrt(1)) # Same as before but now simulate a price in 1 year
    final_spot_simulations.append(final_spot) # Add the simulated spot after one year to our list

Display the list of simulated year-end prices.

In [ ]:
final_spot_simulations[:20]

Plot a histogram of the simulated prices using the same function as earlier.

In [ ]:
plt.hist(final_spot_simulations, bins=20, density=True)
plt.ylabel('density')
plt.xlabel('value')
plt.title('Histogram of the sample')
plt.show()

Calculate the average of the simulated year-end prices.

In [ ]:
np.mean(final_spot_simulations)

The mean of the simulated year-end prices is almost the same as the value obtained from directly applying the annual rate, accounting for the conversion to the effective annual rate.

In [ ]:
rate = np.exp(alpha)-1
1000*(1+rate)

The effective annual rate is:

In [ ]:
rate

### Calculating a features price

A stock price path is simulated, and simultaneously the corresponding futures price is calculated.

In [ ]:
alpha = 0.08
r = 0.02
sigma = 0.25
current_spot = 100

The simulation generates both the spot price path and the corresponding futures prices, stored in path and futures_price lists. There are 12 periods, with each period having $T=1/12$, effectively simulating a path for a year.

In [ ]:
path = [current_spot]
futures_price = [current_spot*np.exp(r)]
for i in range(12):
    z = float(np.random.normal(0,1,1))
    S_i = current_spot * np.exp((alpha-0.5*sigma**2)*(1/12) + z*sigma*np.sqrt(1/12))
    F_i = S_i * np.exp(r*(12-(i+1))/12)
    current_spot = S_i
    path.append(current_spot)
    futures_price.append(F_i)

The plots for both the spot and futures prices are displayed for inspection.

In [ ]:
plt.plot(path)

In [ ]:
plt.plot(futures_price)

In [ ]:
path

In [ ]:
futures_price

Margin is calculated by subtracting the initial futures price from each value along the path. This way we see how much we have of a gain/loss that appears in each period in our account after we went __long__ on a futures contract.

In [ ]:
futures_price[0]

In [ ]:
margin = futures_price - futures_price[0]

In [ ]:
margin

Negative valuse correspond to loss. We find the maximum loss throughout the simulated path.

In [ ]:
np.min(margin)

### Simulating multiple paths and the margin

The same concept is applied here. The previous code is wrapped in a loop to capture the highest loss across all simulations. For each simulation, the margin and its minimum value are calculated. We take that minimum value and save it in the list `worst_list` in the list.

In [ ]:
alpha = 0.08
r = 0.02
sigma = 0.25
spot = 100
n = 10000

worst_list = []
for j in range(n):
    current_spot = spot
    path = [current_spot]
    futures_price = [current_spot*np.exp(r)]
    for i in range(12):
        z = float(np.random.normal(0,1,1))
        S_i = current_spot * np.exp((alpha-0.5*sigma**2)*(1/12) + z*sigma*np.sqrt(1/12))
        current_spot = S_i
        F_i = S_i * np.exp(r*(12-(i+1))/12)
        path.append(current_spot)
        futures_price.append(F_i)
    margin = futures_price - futures_price[0]
    worst = -np.min(margin)
    worst_list.append(worst)

In [ ]:
worst_list[:20]

That's our result! The distribution of worst margin!

In [ ]:
plt.hist(worst_list, bins=10, density=True)
plt.ylabel('density')
plt.xlabel('margin')
plt.title('Distribution of the margin')
plt.show()

In [ ]:
plt.hist(worst_list, bins=20, density=True)
plt.ylabel('density')
plt.xlabel('margin')
plt.title('Distribution of the margin')
plt.show()